In [1]:
!uv add pandas sqlalchemy fastparquet google pandas pandas-gbq pyarrow

Resolved 131 packages in 2ms
Audited 46 packages in 100ms


In [ ]:
import pandas as pd
import sqlalchemy
import fastparquet
from datetime import datetime

from google.cloud import bigquery
from google.oauth2 import service_account

from google.cloud import storage

In [65]:
import sys
import os
import json

cwd = os.getcwd()
secret_file = 'zoomcamp-dt-eng-6fb7bd92bdfb.json'
KEY_PATH = os.path.join(cwd, '.secrets', secret_file)

credentials = service_account.Credentials.from_service_account_file(KEY_PATH)

PROJECT_ID = credentials.project_id
DATASET_ID = 'ny_taxi'
TABLE_ID = 'yellow_tripdata_2024-01'

SKIP_EXISTENT_TABLE = True

In [66]:
uri = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet'

start_date = '2024-01'
end_date = '2024-06'

date_range = pd.date_range(start_date, end_date, freq='MS').strftime('%Y-%m').tolist()
print(date_range)


['2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06']


In [67]:
df = pd.read_parquet(uri)

In [68]:
## Opens connection to Big Query
client = bigquery.Client(project=PROJECT_ID, credentials=credentials)

In [69]:
# Check if the dataset already exists and if not creates it
try:
    client.get_dataset(f'{PROJECT_ID}.{DATASET_ID}')
    print(f"Dataset {PROJECT_ID}.{DATASET_ID} already exists.")
except:
    dataset = bigquery.Dataset(f'{PROJECT_ID}.{DATASET_ID}')
    dataset.lUSation = "US" # Specify the location (e.g., 'US', 'EU')

    # Make the API request to create the dataset
    created_dataset = client.create_dataset(f'{PROJECT_ID}.{DATASET_ID}')
    print(f"Created: {created_dataset}.")

Dataset zoomcamp-dt-eng.ny_taxi already exists.


In [70]:
df.shape

(2964624, 19)

In [71]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [73]:
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE", # Overwrites the table if it already exists
    schema=[
        bigquery.SchemaField("VendorID", bigquery.enums.SqlTypeNames.INTEGER),
        bigquery.SchemaField("passenger_count", bigquery.enums.SqlTypeNames.INTEGER),        
        bigquery.SchemaField("RatecodeID", bigquery.enums.SqlTypeNames.INTEGER)
    ],
)

def load_to_gcs(data, gc_path):
    job = client.load_table_from_dataframe(
        data, gc_path, job_config=job_config
    )
    job.result()

# If flag set as True, skips loading existent table.
if SKIP_EXISTENT_TABLE:
    try: 
        client.get_table(f'{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}')
        print(f'Table "{TABLE_ID}" already exists, skipping load')
    except:
        load_to_gcs(df, f'{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}')
else:
    load_to_gcs(df, f'{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}')
    
    